In [ ]:
pip install ktrain

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
import random
import pandas as pd
import numpy as np
import nltk
import string
from string import digits
from sklearn.model_selection import train_test_split



In [ ]:
class Propaganda:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = self.get_propaganda()        

    def get_propaganda(self):
        if int(self.SUBJprop) <=3 :
            return Propaganda.NEGATIVE
        else:
            return Propaganda.POSITIVE


class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_sentence(self):
        return [x.sentence for x  in self.reviews]

    def get_propaganda(self):
        return [x.propaganda for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, self.reviews))
        negative_shrunk = negative[:len(positive)]
        self.reviews = positive + negative_shrunk
        random.shuffle(self.reviews)

Error: Session cannot generate requests

In [12]:
reviews = []
data  = pd.read_excel('Data/finalDataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['Sentence','SUBJprop'])
# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))

print(df.head())
print("Total Rows:")
print(len(reviews))
print("Total Positive:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))))
print("Total Negative:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))))

                                            Sentence SUBJprop
0  miley and liam fighting false rumors swirl tha...        1
1        are miley cyrus and liam hemsworth fighting        1
2  a new report claims that cyrus and hemsworth h...        1
3  according to the report cyrus 24 and hemsworth...        1
4  but there has reportedly been tension between ...        1
Total Rows:
16774
Total Positive:
3780
Total Negative:
12994


In [10]:
neg_prop = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))
pos_prop = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))

########################################################################################
#split trainig and DevTest dataset
neg_train, neg_devtest  = train_test_split(neg_prop , train_size=0.7, random_state = 42 )
pos_train, pos_devtest = train_test_split(pos_prop , train_size=0.7, random_state = 42 )
########################################################################################
#prepare training dataset
train = neg_train + pos_train
random.shuffle(train)
########################################################################################
#prepare development and test dataset
neg_dev, neg_test = train_test_split(neg_devtest , train_size=0.5, random_state = 42 )
pos_dev, pos_test = train_test_split(pos_devtest , train_size=0.5, random_state = 42 )

dev = neg_dev + pos_dev
random.shuffle(dev)

test = neg_test + pos_test
random.shuffle(test)
########################################################################################
print("Total Train:")
print(len(train))
print("Positive:")
print(len(pos_train))
print("Negative:")
print(len(neg_train))

print("\nTotal Development:")
print(len(dev))
print("Positive:")
print(len(pos_dev))
print("Negative:")
print(len(neg_dev))

print("\nTotal Test:")
print(len(test))
print("Positive:")
print(len(pos_test))
print("Negative:")
print(len(neg_test))

Total Train:
11741
Positive:
2646
Negative:
9095

Total Development:
2516
Positive:
567
Negative:
1949

Total Test:
2517
Positive:
567
Negative:
1950


In [11]:
train_container = ReviewContainer(train)
train_container.evenly_distribute()

train_sentence = train_container.get_sentence()   
train_propaganda = train_container.get_propaganda() 

development_sentence = [x.sentence for x in dev]
development_propaganda = [x.propaganda for x in dev]

test_sentence = [x.sentence for x in test]
test_propaganda = [x.propaganda for x in test]
print("Toalt rows after balance training dataset:")
print(len(train_sentence))

print("Positive Propaganda:")
print(train_propaganda.count(Propaganda.POSITIVE))

print("Negative Propaganda:")
print(train_propaganda.count(Propaganda.NEGATIVE))

Toalt rows after balance training dataset:
5292
Positive Propaganda:
2646
Negative Propaganda:
2646


Using the Bert

In [ ]:
(train_sentence, train_propaganda), (test_sentence, test_propaganda), preproc = text.texts_from_df(df,'Sentence',label_columns=['SUBJprop'],maxlen=400,
max_features=1000,preprocess_mode='bert')

In [30]:

model = text.text_classifier('bert', (train_sentence, train_propaganda) , preproc=preproc)


Is Multi-Label? False
maxlen is 400
done.


In [ ]:

learner = ktrain.get_learner(model, 
                             train_data=(train_sentence, train_propaganda), 
                             val_data=(test_sentence, test_propaganda), 
                             batch_size=32)

In [ ]:
p = ktrain.get_predictor(learner.model, preproc)


Error: Session cannot generate requests

In [ ]:
predictor.save('Model/Bert')


Error: Session cannot generate requests

In [ ]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']
predictor.predict(data)


Error: Session cannot generate requests